In [7]:
from dataset import PolarDecDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch
from models.wrappers.mamba_32bits import MambaPolarDecoder

In [8]:
N = 32
CONFIG_NO = 7

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

### Dataset

In [10]:
dataset = PolarDecDataset(snr_db=10, num_samples=100000, seq_length=N)
test_set = PolarDecDataset(snr_db=10, num_samples=3200, seq_length=N)


In [11]:
train_dataloader = DataLoader(dataset, batch_size = 32)
test_dataloader = DataLoader(test_set, batch_size = 32)

## Model

In [12]:
model = MambaPolarDecoder(
    d_model=32,
    num_layer_encoder=1,
    num_layers_bimamba_block=8,
    seq_len=N,
    d_state=16,
    d_conv=4,
    expand=2
).to(device)
model

MambaPolarDecoder(
  (discrete_embedding): Embedding(2, 32)
  (linear_embedding1): Linear(in_features=1, out_features=32, bias=True)
  (linear_input_layer): Linear(in_features=64, out_features=32, bias=True)
  (encoder_layers): ModuleList(
    (0): BiMambaEncoder(
      (layers): ModuleList(
        (0-7): 8 x BiMambaBlock(
          (pre_ln_f): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
          (mamba_f): Mamba(
            (in_proj): Linear(in_features=32, out_features=128, bias=False)
            (conv1d): Conv1d(64, 64, kernel_size=(4,), stride=(1,), padding=(3,), groups=64)
            (act): SiLU()
            (x_proj): Linear(in_features=64, out_features=34, bias=False)
            (dt_proj): Linear(in_features=2, out_features=64, bias=True)
            (out_proj): Linear(in_features=64, out_features=32, bias=False)
          )
          (post_ln_f): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
          (ffn_f): Sequential(
            (0): Linear(in_feature

In [8]:
# Run/Change if loading an old model only

model_path = "./checkpoints/config_3/model_epoch_29.pt"
checkpoint = torch.load(model_path, map_location=device)

state_dict = checkpoint.get("model_state_dict", checkpoint.get("state_dict", checkpoint))
model.load_state_dict(state_dict)

<All keys matched successfully>

## Minor modification to the Loss Function: Calculates loss only at non frozen positions

In [14]:
def calculate_loss(frozen_bit_prior, target_vector, predicted_vector,  reliable_only=False):
    """
    frozen_bit_prior: tensor of shape (seq_len,) with 1 for frozen, 0 for message bits
    target_vector: tensor of shape (seq_len,)
    predicted_vector: tensor of shape (seq_len,)
    loss_fn: PyTorch loss function
    """

    if reliable_only: 
     mask = (frozen_bit_prior != 1) 
     target_vector = target_vector[mask]
     predicted_vector = predicted_vector[mask]

    #print("target vector:" ,target_vector[:32], "\n")
    #print("pred vector:" ,predicted_vector[:32])

   # print(f"Length of reliable bits: {len(reliable_target)}")
    loss_fn = torch.nn.BCEWithLogitsLoss()

    return loss_fn(predicted_vector, target_vector)

## Sanity Check

In [22]:
llr, frozen_tensor, snr_tensor, target_tensor= next(iter(train_dataloader))
ip1 = llr.float().to(device)
ip2 = frozen_tensor.int().to(device)
ip3 = snr_tensor.float().to(device)

predicted = model(ip1, ip2, ip3) #works

loss = calculate_loss(ip2, target_tensor.to(device), predicted.to(device)) #works

#print(f"Channel Observation Vector: {channel_tensor}\n\n")


#print(f"Channel Observation Vector: {ip1.shape}\nFrozen Tensor: {ip2.shape}\n")
#print(f"Predicted Channel Input Vector(logits): {predicted.shape}\n\n")

#print(f"Predicted (sigmoid): {torch.sigmoid(predicted)}\n\n")
pred = (torch.sigmoid(predicted) > 0.5).long()[0]

print(f"Predicted bits:{''.join(map(str, pred.cpu().tolist()))}\n")
print(f"Actual bits: {''.join(str(int(i)) for i in target_tensor[0])}\n")
print(f"Loss: {loss}")

Predicted bits:10110100100011101111000011100000

Actual bits: 10000010100011101111000011100000

Loss: 0.4178037941455841


In [23]:

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=0)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', 
                                                        factor=0.5, patience=3)


In [24]:
def train_one_epoch(epoch_index):

    running_loss = 0
    last_loss = 0


    for i, data in enumerate(train_dataloader):
        channel_tensor, frozen_tensor, snr_tensor, target_tensor = data
        ip1 = channel_tensor.float().to(device)
        ip2 = frozen_tensor.int().to(device)
        ip3 = snr_tensor.float().to(device)
        op = target_tensor.to(device)
        optimizer.zero_grad()
        outputs = model(ip1,ip2 ,ip3 ).to(device)

    #    B, L, C = outputs.shape
    #    output_logits = outputs.view(B*L, C).to(device)
    #    target_flattened = shifted.view(B*L).to(device).long()


    #    loss = loss_fn(output_logits, target_flattened)
        
        loss = calculate_loss(ip2, op, outputs)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        running_loss += loss.item()
        if i%1000 == 999:
            last_loss = running_loss/1000
            print('  batch {} loss: {}\n'.format(i + 1, last_loss))
          #  print(f"Predictions look currently like: {outputs[:32]}\n\n")
            running_loss = 0.
    return last_loss


In [25]:
def train( epochs=50):

    best_vloss = 1_000_000.

    for epoch in range(epochs):
     print('EPOCH {}:'.format(epoch + 1))
 
   
     model.train(True)
     avg_loss = train_one_epoch(epoch)


     running_vloss = 0.0
    
     model.eval()

   
     with torch.no_grad():
        for i, vdata in enumerate(test_dataloader):
            vchannel_tensor, vfrozen_tensor, vsnr_tensor, vtarget_tensor = vdata
            voutputs = model(vchannel_tensor.float().to(device), vfrozen_tensor.int().to(device), vsnr_tensor.float().to(device))
          #  B, L, C = voutputs.shape
          #  vloss = loss_fn(voutputs.view(B*L, C).to(device), vlabels.view(B*L).to(device))
            
            vloss = calculate_loss(vfrozen_tensor.to(device), vtarget_tensor.to(device), voutputs.to(device))
            running_vloss += vloss

     avg_vloss = running_vloss / (i + 1)
     print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    
     scheduler.step(avg_vloss)

    
     if avg_vloss < best_vloss:
      
    
      best_vloss = avg_vloss
      model_path = f'./checkpoints/config_{CONFIG_NO}/model_epoch_{epoch}.pt'
      torch.save({
         "comments": "Removed the snr as input entirely. (even if used in future, use as snr linear, not in db)",
    'model_config': {
        "d_model": model.d_model,
        "num_layer_encoder": model.num_layer_encoder,
        "num_layers_bimamba_block": model.num_layers_bimamba_block,
        "seq_len": model.seq_len,
        "d_state": model.d_state,
        "d_conv": model.d_conv,
        "expand": model.expand,
    },
    'epoch': epoch + 1,
    'train_loss': avg_loss,
    'val_loss': avg_vloss,
    'state_dict': model.state_dict()
}, model_path)


     
    print("Training completed. Model available to use")





In [26]:
train(epochs=50)

EPOCH 1:
  batch 1000 loss: 0.16876117210835218

  batch 2000 loss: 0.13451092935353517

  batch 3000 loss: 0.12998631876707076

LOSS train 0.12998631876707076 valid 0.13152478635311127
EPOCH 2:
  batch 1000 loss: 0.1293222303763032

  batch 2000 loss: 0.12926761538535356

  batch 3000 loss: 0.12875927066802978

LOSS train 0.12875927066802978 valid 0.12942537665367126
EPOCH 3:
  batch 1000 loss: 0.1291515857130289

  batch 2000 loss: 0.12859168649464844

  batch 3000 loss: 0.12851607444137333

LOSS train 0.12851607444137333 valid 0.12577594816684723
EPOCH 4:
  batch 1000 loss: 0.12914862057566642

  batch 2000 loss: 0.1288262109979987

  batch 3000 loss: 0.12876178611814976

LOSS train 0.12876178611814976 valid 0.13512858748435974
EPOCH 5:
  batch 1000 loss: 0.1296358618363738

  batch 2000 loss: 0.1283669722750783

  batch 3000 loss: 0.12912962184101343

LOSS train 0.12912962184101343 valid 0.1289234757423401
EPOCH 6:
  batch 1000 loss: 0.12744715800881387

  batch 2000 loss: 0.129465